---
title: SMC With Translators
date: "January 11, 2024"
abstract: "WIP"
callout-appearance: simple
---

In [ ]:
import genjax
import jax
import jax.numpy as jnp
import jax.tree_util as jtu
from genjax import ChoiceMap
from genjax.incremental import NoChange, UnknownChange, diff
from genjax.inference import smc
from genjax.inference.translator import extending_trace_translator
from genjax.typing import typecheck


# Model
@genjax.unfold_combinator(max_length=10)
@genjax.static
def chain(z_prev):
    z = genjax.normal(z_prev, 1.0) @ "z"
    _x = genjax.normal(z, 1.0) @ "x"
    return z


# Proposal and step-indexed translator.
# This function can be freely used inside scan'd code.
def get_translator(t, obs):
    # The proposal signature for this translator is:
    # (obs_chm, prev_particle, *proposal_args)
    @genjax.static
    @typecheck
    def proposal(obs_chm: ChoiceMap, prev_particle: ChoiceMap, *args):
        masked_x = obs_chm[t, "x"]  # from idx choice map
        x = masked_x.unmask()  # will throw if invalid under checkify.
        z = (
            genjax.normal(x, 0.01) @ "z"
        )  # tight variance to make sure everything is working
        return z

    # The translator allows a bijection on the space of choice maps.
    # We utilize this degree of freedom to change the address representation
    def choice_map_forward(proposal_choices):
        return genjax.indexed_choice_map(t, proposal_choices)

    # The inverse just removes the dynamic index.
    # Unmask is safe under `checkify` - it will throw if
    # the mask is invalid.
    def choice_map_inverse(transformed_choices):
        return transformed_choices[t].unmask()

    translator = extending_trace_translator(
        (diff(t, UnknownChange), diff(0.0, NoChange)),
        proposal,
        (),
        obs,
        choice_map_forward,
        choice_map_inverse,
        # Dynamically checks the bijection, and registers
        # the error with `jax.checkify` if it fails.
        check_bijection=True,
    )
    return translator


# Observations, indexed for each time step.
obs = genjax.indexed_choice_map(
    [0, 1, 2, 3],
    genjax.choice_map({"x": jnp.array([1.0, 2.0, 3.0, 4.0])}),
)


# SMC with translator in the loop.
def extending_smc(key, obs, init_state):
    # Initialize a particle collection (SMCState)
    _index_sel = genjax.indexed_select(0)
    obs_init = obs.slice(0)
    key, sub_key = jax.random.split(key)
    smc_state = smc.smc_initialize(chain, n_particles=5).apply(
        sub_key, obs_init, (0, init_state)
    )

    # Now, scan over the observations, and use the translator
    # to perform SMC extend steps.
    obs = jtu.tree_map(lambda v: v[1:], obs)

    def _inner(carry, xs):
        smc_state, t = carry
        obs_slice = xs
        t = t + 1
        translator = get_translator(t, obs_slice)
        sub_key = jax.random.fold_in(key, t)

        # The SMC mini-language has an update move
        # which accepts a translator and uses it
        # to perform the move:
        # * proposing from proposal
        # * translating proposal choices (induces Jacobian term)
        # * updating the previous particle trace using translated choices
        #   & new obs
        # * computing correct weight (model - proposal - jacobian_log_abs_det)
        smc_state = smc.SMCExtendUpdate(translator).apply(
            sub_key,
            smc_state,
        )

        return (smc_state, t), (smc_state,)

    (final_state, _), (stacked,) = jax.lax.scan(
        _inner,
        (smc_state, 0),
        obs,
    )
    return final_state, stacked

In [ ]:
from jax.experimental.checkify import checkify

key = jax.random.PRNGKey(314159)

# Use the console to enforce checkify -- this enables
# runtime checks throughout GenJAX.
console = genjax.console(enforce_checkify=True)
with console:
    err, (final_state, stacked) = jax.jit(checkify(extending_smc))(key, obs, 0.0)
    err.throw()

console.print(final_state.particles.get_choices().slice(1)[3, "z"].unmask())